# Create Anonymized Data

The Goal of this notebook is to create synthetic data from harmonization purposes. Initially I will follow the easiest path forward by simply selecting 5 random values from each column. This will preserve the integrity of the data it render it useless for any analysis purposes. I will manually remove any location information. At a later step. 

In [1]:
import pandas as pd
import fsspec
from pathlib import Path

import numpy as np

In [2]:
fs = fsspec.filesystem("")

In [3]:
avail_studies = [f.split('/')[-1] for f in fs.ls('../data/') if f.split('/')[-1][0] != '.']
avail_studies

['Europe_CH_SIB', 'H3Africa']

Version 1 just picked 10 rows at random but this risks getting no values if there are lots on None's. I will instead pick 10 random values from the set of the values. If the set is less than 10 I will pad it with None's. 

In [4]:
completed = []
for study in avail_studies:
    if fs.isdir(f'../data/{study}/csv/'):
        csv_files = [f.split('/')[-1] for f in fs.ls(f'../data/{study}/csv/') if f.split('/')[-1][0] != '.']
        for file in csv_files:
            df = pd.read_csv(f'../data/{study}/csv/{file}', low_memory=False).iloc[:,1:]
            data = {}
            for col in df.columns:
                set_of_values = list(set(df[col].dropna()))
                n = len(set_of_values)
                if n >= 10:
                    data[col] = [set_of_values[i] for i in np.random.randint(low = 0, high = n, size = 10)]
                else:
                    data[col] = (set_of_values + 10 * [None])[:10]
            df_synth = pd.DataFrame.from_dict(data)
            fs.mkdirs(f'../data/{study}/synthetic', exist_ok = True)
            df_synth.to_csv(f'../data/{study}/synthetic/{file}')
            del df
            del df_synth

## an important next step is to manually go through the generated synthetic data and ensure no location info is present!